In [1]:
import torch
print(torch.__version__)
print('CUDA available:', torch.cuda.is_available())

ImportError: /home/christianl/miniconda3/envs/remote_training/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so: undefined symbol: iJIT_NotifyEvent

In [ ]:
%run '/home/christianl/Zhang-Lab/Zhang Lab Code/Boilerplate/model_boilerplate_remote.py'

In [ ]:
loaded_model = torch.load('/home/christianl/Zhang-Lab/Zhang Lab Data/Saved models/RNN/signaling_model.v1.pt')